# Semantic Search

Reference and more details: https://www.sbert.net/examples/applications/semantic-search/README.html

In [1]:
import json
import os

output_dir = "output"
json_files = [file_name for file_name in os.listdir(output_dir) if file_name.endswith(".json")]

corpus = []

for json_file in json_files:
    with open(os.path.join(output_dir,json_file)) as json_input:
        json_content = json.load(json_input)
        for item in json_content:
            url = item["url"]
            text_paragraphs = item["text"].split("\n")
            text_paragraphs = [(para, json_file[:-5], item["topic"], item["url"]) for para in text_paragraphs if len(para.split())>10 \
                               and (para, json_file[:-5], item["topic"], item["url"]) not in corpus]
            corpus += text_paragraphs
            

In [2]:
corpus[0:5]

[('На інтерактивній мапі відображаються тільки діючі пункти пропуску. Пункти пропуску, що тимчасово закриті відповідно до розпорядження КМУ від 13.03.2020 року №288-р, на мапі не відображаються! (dpsu.gov.ua)',
  'Польща',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/ab8ea9a2a8f945f1b784e910d566ca79'),
 ('Telegram-канал Dorohusk - актуальна інформація про стан кордону, пошук трансфера, інша допомога',
  'Польща',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/ab8ea9a2a8f945f1b784e910d566ca79'),
 ('В’їзд до Польщі можливий на підставі будь-якого документу, що посвідчує особу (закордоний паспорт, внутрішнього посвідчення особи, або свідоцтва про народження для неповнолітніх)',
  'Польща',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/ab8ea9a2a8f945f1b784e910d566ca79'),
 ('Якщо ви перетинаєте кордон на машині потрібно мати Зелену карту. Її можна оформити в Приват 24 або в пункті надання допомоги одразу після перетину кордону. На місяць Зелена карта п

In [3]:
from sentence_transformers import SentenceTransformer

corpus_sentences = [item[0] for item in corpus]

embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

corpus_embeddings = embedder.encode(corpus_sentences, convert_to_tensor=True)

In [4]:
# Save corpus_embeddings to a file to be able to load later
import pickle

with open("corpus_embeddings.pkl", "wb") as embeddings:
    pickle.dump({'corpus': corpus, 'embeddings': corpus_embeddings}, embeddings)


# To load the embeddings later from the file instead of creating from scratch
'''
import pickle
from sentence_transformers import SentenceTransformer

with open("corpus_embeddings.pkl", "rb") as embeddings:
    data = pickle.load(embeddings)
    corpus = data['corpus']
    corpus_sentences = [item[0] for item in corpus]
    corpus_embeddings = data['embeddings']
    
    embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

    corpus_embeddings = embedder.encode(corpus_sentences, convert_to_tensor=True)
'''

In [7]:
from sentence_transformers import util
import torch

queries = ["Освіта"]  # Education

# Find the closest 20 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(20, len(corpus_sentences))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("• Query:", query)
    print("• Top 20 most similar sentences in corpus:\n")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx][0])
        print("Read more:", corpus[idx][1], "--", corpus[idx][2], corpus[idx][3])
        print("[Score: {:.2f}]".format(score), "\n")

• Query: Освіта
• Top 20 most similar sentences in corpus:

До яких державних та неурядових органів я можу звернутися за додатковою інформацією про освіту?
Read more: Нідерланди -- Освіта та мовні курси https://uahelpinfo.notion.site/7f65b976009e4ec2b402c89d4f21702a
[Score: 0.64] 

До яких державних та неурядових органів я можу звернутися за додатковою інформацією про освіту?
Read more: Португалія -- Освіта та мовні курси https://uahelpinfo.notion.site/3dd45be7c4294f60ae7f8a63f641eedb
[Score: 0.64] 

До яких державних та неурядових органів я можу звернутися за додатковою інформацією про освіту?
Read more: Угорщина -- Освіта та мовні курси https://uahelpinfo.notion.site/492c534b8b654ad687b006b5627e5d5a
[Score: 0.64] 

До яких державних та неурядових органів я можу звернутися за додатковою інформацією про освіту?
Read more: Швеція -- Освіта та мовні курси https://uahelpinfo.notion.site/14c4eb3753f4441aab19e2abb8ab7832
[Score: 0.64] 

До яких державних та неурядових органів я можу звернут